<a href="https://colab.research.google.com/github/Akash-an/LLM/blob/master/mdt_cc_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install datasets

In [ ]:
from transformers import TFGPT2LMHeadModel, AutoConfig
from transformers import AutoTokenizer
import tensorflow as tf

from datasets import Dataset

In [ ]:
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
gpt_tokenizer.vocab_size

50257

In [ ]:
htmlstring = """
 <assign:variable>
 <p>
                    <name>AKA_PM_TD_ENABLED</name>
                               </p>
                    <value>true</value>

                </assign:variable>
                """
# htmlstring= '''
# <match:region.number value="36714 36715 36704 36705">
#       <network:dns.rewrite>
#         <status>on</status>
#         <rule>/taa-customer-origin.s3.ap-south-1.amazonaws.com/52.219.64.3/</rule>
#       </network:dns.rewrite>
#     </match:region.number>'''

In [ ]:
enc = gpt_tokenizer(htmlstring)
enc

{'input_ids': [198, 1279, 562, 570, 25, 45286, 29, 198, 1279, 79, 29, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1279, 3672, 29, 10206, 32, 62, 5868, 62, 21016, 62, 1677, 6242, 30465, 3556, 3672, 29, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 7359, 79, 29, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1279, 8367, 29, 7942, 3556, 8367, 29, 628, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 7359, 562, 570, 25, 45286, 29, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
for t in enc['input_ids']:
    if t==220:
        continue
    print(gpt_tokenizer.decode(t))



 <
ass
ign
:
variable
>


 <
p
>


 <
name
>
AK
A
_
PM
_
TD
_
EN
AB
LED
</
name
>


 </
p
>


 <
value
>
true
</
value
>



 </
ass
ign
:
variable
>




### Create raw_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

directory = '/content/drive/MyDrive/metadata/'

In [ ]:
def load_data(file_names,folder,limit=None):

    loaded_data = {'mdt':[], 'file':[]}

    i =0
    for file_name in file_names:
        with open(directory+folder+file_name) as f:
            # print(file_name)
            file_data = f.read()
            loaded_data['mdt'].append(file_data)
            loaded_data['file'].append(file_name)

        i += 1
        if limit is not None:
            if i>limit:
                break
    return loaded_data

In [ ]:
arl_files = os.listdir(directory+'arl-test-properties/')
pearl_files = os.listdir(directory+'product-mdt/pearl.data/')
hd_files = os.listdir(directory+'product-mdt/hd.data/')

In [ ]:
raw_dataset = {}

raw_dataset['arls'] = load_data(arl_files,'arl-test-properties/',70)
raw_dataset['pearls'] = load_data(pearl_files,'product-mdt/pearl.data/')
raw_dataset['hds'] = load_data(hd_files,'product-mdt/hd.data/')


In [ ]:
# !!! dataset deciding variable
data_to_be_included = ['pearls','hds','arls']

In [ ]:
complete_dataset = {'mdt':[], 'file':[]}
for data in data_to_be_included:
    complete_dataset['mdt'] += raw_dataset[data]['mdt']
    complete_dataset['file'] += raw_dataset[data]['file']

In [ ]:
len(complete_dataset['file'])

155

In [ ]:
hf_raw_dataset = Dataset.from_dict({'file':complete_dataset['file'], 'mdt':complete_dataset['mdt']})

In [ ]:
split_dataset = hf_raw_dataset.train_test_split(test_size=0.1)

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'mdt'],
        num_rows: 139
    })
    test: Dataset({
        features: ['file', 'mdt'],
        num_rows: 16
    })
})

In [ ]:
hf_raw_dataset[:2]['file']

['bss_pearls.xml', 'amos_pearls.xml']

## Train the tokenizer

In [ ]:
tokenizer = gpt_tokenizer.train_new_from_iterator(split_dataset['train']['mdt'], 50257)

In [ ]:
htmlstring = """
 <assign:variable>
 <p>
                    <name>AKA_PM_TD_ENABLED</name>
                               </p>
                    <value>true</value>

                </assign:variable>
                """

In [ ]:
enc = tokenizer(htmlstring)
enc

{'input_ids': [199, 263, 392, 26, 374, 30, 199, 263, 80, 30, 487, 263, 347, 30, 615, 63, 607, 63, 1000, 63, 918, 274, 347, 30, 7428, 307, 80, 30, 487, 263, 358, 30, 276, 274, 358, 30, 3016, 307, 392, 26, 374, 30, 5524], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
for t in enc['input_ids']:
    print(tokenizer.decode(t))



 <
assign
:
variable
>


 <
p
>

                   
 <
name
>
AKA
_
PM
_
TD
_
ENABLED
</
name
>

                              
 </
p
>

                   
 <
value
>
true
</
value
>


               
 </
assign
:
variable
>

                


### raw dataset -> tokenized dataset

In [ ]:
def tokenize(elements):
    outputs = tokenizer(
        elements['mdt'],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []
    for inp_ids, len in zip(outputs['input_ids'], outputs['length']):
        if len==128:
            input_batch.append(inp_ids)

    return {'input_ids':input_batch}

tokenized_dataset = split_dataset['train'].map(tokenize, remove_columns=['file','mdt'], batched=True)
tokenized_dataset_test = split_dataset['test'].map(tokenize, remove_columns=['file','mdt'], batched=True)

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
# for t in tokenized_dataset[4]['input_ids']:
#     print(tokenizer.decode(t))

tokenized_dataset[4]

## model training

In [ ]:
from transformers import TFGPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    'gpt2',
    vocab_size=len(tokenizer),
    n_ctx=64,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124439808 
 r)                                                              
                                                                 
Total params: 124,439,808
Trainable params: 124,439,808
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
## load the model from hugging face

model = TFGPT2LMHeadModel.from_pretrained("akash0/mdt-cc-1.2")
tokenizer = AutoTokenizer.from_pretrained("akash0/mdt-cc-1.2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at akash0/mdt-cc-1.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16
)
tf_test_dataset = model.prepare_tf_dataset(
    tokenized_dataset_test,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16
)

In [ ]:
#prepare for training. Define a custom(decaying) learning rate.

from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
        init_lr=5e-5,
        num_warmup_steps=100,
        num_train_steps=num_train_steps,
        weight_decay_rate=0.01
)
model.compile(optimizer=optimizer)


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="mdt-cc-1.4", tokenizer=tokenizer)

/content/mdt-cc-1.4 is already a clone of https://huggingface.co/akash0/mdt-cc-1.4. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.fit(tf_train_dataset,validation_data=tf_test_dataset, callbacks=[callback],epochs=1)

5846/5846 [==============================] - 1234s 211ms/step - loss: 1.1120 - val_loss: 1.8227


In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, device=0
)

In [ ]:
htmlstring = '''
<transform>
        <hmac>
            <key>abc123</key>
            <algorithm>SHA256</algorithm>'''

print(pipe(htmlstring, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



<transform>
        <hmac>
            <key>abc123</key>
            <algorithm>SHA256</algorithm>
          </hmac>
          </transform>
        </assign:variable>
        <assign:variable>
          <name>


In [ ]:
pipe(htmlstring, num_return_sequences=1)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'\n<transform>\n        <hmac>\n            <key>abc123</key>\n            <algorithm>SHA256</algorithm>\n          </hash>\n        </transform>\n        <transform>\n          <direction>off</algorithm>\n        <hidden>'